<a href="https://colab.research.google.com/github/anuradha1105/Autogluon/blob/main/colabs/01_ieee-fraud-autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ==== System info ====
!nvidia-smi -L || echo "CPU-only runtime"
!python --version

# ==== Install AutoGluon and Kaggle ====
!pip install -U pip
!pip install -U "autogluon>=1.0" kaggle

# ==== Ensure folders exist ====
import os
for d in ["data","artifacts"]:
    os.makedirs(d, exist_ok=True)
print("Setup complete. Remember: Edit ▸ Notebook settings ▸ uncheck 'Omit code cell output when saving'.")


GPU 0: Tesla T4 (UUID: GPU-42c4c1a8-960a-1b90-ab3a-61fce47e18ae)
Python 3.12.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 97.3 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.31.7 requires filelock>=3.15, but you have filelock 3.13.1 which is incompatible.


Setup complete. Remember: Edit ▸ Notebook settings ▸ uncheck 'Omit code cell output when saving'.


In [4]:
# ==== System info ====
!nvidia-smi -L || echo "CPU-only runtime"
!python --version

# ==== Install (with safe pins) ====
# - numpy<2.1 avoids import reload issues for some libs
# - filelock>=3.15 satisfies pytensor's requirement
!pip install -U "pip>=23.3" -q
!pip install -U "numpy<2.1" "filelock>=3.15" "autogluon>=1.0" kaggle --no-warn-conflicts -q

# ==== Ensure folders ====
import os
for d in ["data","artifacts"]:
    os.makedirs(d, exist_ok=True)
print("Setup complete.")


GPU 0: Tesla T4 (UUID: GPU-42c4c1a8-960a-1b90-ab3a-61fce47e18ae)
Python 3.12.12
Setup complete.


In [5]:
import numpy, importlib
from autogluon.tabular import TabularPredictor
print("NumPy:", numpy.__version__)
print("AutoGluon OK")


NumPy: 2.1.2
AutoGluon OK


In [1]:
import torch, autogluon
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("AutoGluon:", autogluon.__version__)


Torch: 2.6.0+cu124
CUDA available: True


AttributeError: module 'autogluon' has no attribute '__version__'

In [2]:
!pip install -U --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp312-cp312-linux_x86_64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp312-cp312-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp312-cp312-linux_x86_64.whl.metadata (6.6 kB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_nvrt

In [1]:
import torch, autogluon
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("AutoGluon:", autogluon.__version__)


Torch: 2.6.0+cu124
CUDA available: True


AttributeError: module 'autogluon' has no attribute '__version__'

In [2]:
import autogluon.core as ag
print("AutoGluon version:", ag.__version__)


AutoGluon version: 1.4.0


In [ ]:
!pip install -U --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 129.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 143.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 36.2 MB/s  0:00:09
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.6 MB/s  0:00:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 49.9 MB/s  0:00:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 63.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 66.3 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 71.5 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 70.7 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 144.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 155.9 MB/

In [ ]:
!pip install autogluon --quiet


In [ ]:
!nvidia-smi


Tue Oct 14 08:10:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
# ==== Kaggle API setup ====
from google.colab import files
from pathlib import Path
import os

print("Upload kaggle.json (Kaggle ▸ Account ▸ Create New API Token)")
uploaded = files.upload()  # choose kaggle.json from your computer

Path("/root/.kaggle").mkdir(parents=True, exist_ok=True)
with open("/root/.kaggle/kaggle.json","wb") as f:
    f.write(uploaded['kaggle.json'])
os.chmod("/root/.kaggle/kaggle.json", 0o600)

!kaggle competitions list -s fraud | head -n 5


Upload kaggle.json (Kaggle ▸ Account ▸ Create New API Token)


Saving kaggle.json to kaggle (5).json


KeyError: 'kaggle.json'

In [17]:
!kaggle competitions list -s ieee | head -n 5



Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 441, in authenticate
    self._load_config(config_data)
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 492, in _load_config
    raise ValueError('Error: Missing %s in configuration.' % item)
ValueError: Error: Missing username in configuration.


In [19]:
!rm -rf /root/.kaggle


In [20]:
from google.colab import files
from pathlib import Path
import os, json

print("Upload the new kaggle.json (just downloaded from Kaggle Account page)…")
uploaded = files.upload()

filename = list(uploaded.keys())[0]  # automatically detect uploaded filename

Path("/root/.kaggle").mkdir(parents=True, exist_ok=True)
with open("/root/.kaggle/kaggle.json", "wb") as f:
    f.write(uploaded[filename])
os.chmod("/root/.kaggle/kaggle.json", 0o600)

# show what keys exist without exposing secrets
cfg = json.load(open("/root/.kaggle/kaggle.json"))
print("✅ Keys in file:", list(cfg.keys()))


Upload the new kaggle.json (just downloaded from Kaggle Account page)…


Saving kaggle.json to kaggle (6).json
✅ Keys in file: ['username', 'key']


In [21]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/root/.kaggle"

!kaggle config view
!kaggle datasets list -s titanic | head -n 5


Configuration values from /root/.kaggle
- username: anuradhasrivastav
- path: None
- proxy: None
- competition: None
ref                                  title                                                size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------  ---------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
heptapod/titanic                     Titanic                                             11090  2017-05-16 08:14:22.210000         131450       1741  0.7058824        
brendan45774/test-file               Titanic dataset                                     11514  2021-12-02 16:11:42.367000         201915       1581  1.0              
yasserh/titanic-dataset              Titanic Dataset                                     22564  2021-12-24 14:53:06.913000         220178        677  1.0              


In [22]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/root/.kaggle"

!kaggle competitions list -s ieee | head -n 10


No competitions found


In [26]:
!rm -rf ~/.cache/kaggle
!kaggle competitions list -s ieee | head -n 10


No competitions found


In [27]:
!rm -rf ~/.cache/kaggle
!kaggle --version
!pip install --upgrade kaggle
!kaggle --version


Kaggle API 1.7.4.5
Kaggle API 1.7.4.5


In [28]:
!kaggle competitions download -c ieee-fraud-detection -p data

403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/competitions/data/download-all/ieee-fraud-detection


In [29]:
# ---- Kaggle download with graceful fallback to Google Drive or manual upload ----
import os, subprocess, json
from google.colab import drive, files

os.makedirs("data/ieee", exist_ok=True)

def try_kaggle_download():
    try:
        # Try direct download via Kaggle CLI
        print("Trying Kaggle CLI download...")
        subprocess.check_call(
            ["kaggle","competitions","download","-c","ieee-fraud-detection","-p","data"]
        )
        return True
    except subprocess.CalledProcessError as e:
        print("Kaggle download failed:", e)
        return False

ok = try_kaggle_download()

# If Kaggle blocks (403), use Google Drive zip if present; else prompt for manual upload
if not ok:
    print("🔁 Falling back: Google Drive or manual upload of ieee-fraud-detection.zip")

    # 1) Mount Drive and look for a zip you downloaded via the website
    drive.mount("/content/drive", force_remount=True)
    candidates = [
        "/content/drive/My Drive/ieee-fraud-detection.zip",
        "/content/drive/MyDrive/ieee-fraud-detection.zip",
        "/content/drive/My Drive/Kaggle/ieee-fraud-detection.zip",
        "/content/drive/MyDrive/Kaggle/ieee-fraud-detection.zip",
    ]

    found = None
    for p in candidates:
        if os.path.exists(p):
            found = p
            break

    if found:
        print(f"✅ Found zip in Drive: {found}")
        !_cp "{found}" data/
    else:
        print("📥 Upload ieee-fraud-detection.zip from your computer (downloaded from Kaggle → Data tab)")
        uploaded = files.upload()  # choose the big zip here
        # Keep whatever filename you uploaded

# Unzip if the archive is present
if os.path.exists("data/ieee-fraud-detection.zip"):
    print("Unzipping ieee-fraud-detection.zip → data/ieee ...")
    !unzip -qo data/ieee-fraud-detection.zip -d data/ieee
    !ls -lh data/ieee | head -n 20
else:
    print("❗No zip found yet. If needed, download from:")
    print("  https://www.kaggle.com/competitions/ieee-fraud-detection/data")
    print("Then place ieee-fraud-detection.zip in Google Drive (paths checked above) or upload here.")


Trying Kaggle CLI download...
Kaggle download failed: Command '['kaggle', 'competitions', 'download', '-c', 'ieee-fraud-detection', '-p', 'data']' returned non-zero exit status 1.
🔁 Falling back: Google Drive or manual upload of ieee-fraud-detection.zip


MessageError: Error: credential propagation was unsuccessful

In [30]:
kaggle competitions download -c [COMPETITION]


SyntaxError: invalid syntax (ipython-input-4087209419.py, line 1)

In [31]:
# ------------------------------------------------------------------
# IEEE Fraud dataset: Kaggle download OR fallback sample generator
# ------------------------------------------------------------------
import os, subprocess, pandas as pd, numpy as np
from sklearn.datasets import make_classification
os.makedirs("data/ieee", exist_ok=True)

print("🧩 Trying Kaggle download...")
try:
    subprocess.check_call(
        ["kaggle", "competitions", "download", "-c", "ieee-fraud-detection", "-p", "data"]
    )
    os.system("unzip -qo data/ieee-fraud-detection.zip -d data/ieee")
    print("✅ Kaggle data downloaded.")
except Exception as e:
    print("⚠️ Kaggle download failed:", e)
    print("➡️  Generating small sample dataset instead (for demo only).")
    X, y = make_classification(
        n_samples=5000, n_features=30, n_informative=10, weights=[0.96, 0.04], random_state=42
    )
    df = pd.DataFrame(X, columns=[f"V{i}" for i in range(1,31)])
    df["TransactionID"] = np.arange(1000000, 1000000+len(df))
    df["isFraud"] = y
    df.to_csv("data/ieee/train_transaction.csv", index=False)
    df.sample(frac=0.5).drop("isFraud", axis=1).to_csv("data/ieee/test_transaction.csv", index=False)
    pd.DataFrame({
        "TransactionID": range(1000000, 1000000+len(df)//2),
        "isFraud": 0
    }).to_csv("data/ieee/sample_submission.csv", index=False)

!ls -lh data/ieee | head -n 10


🧩 Trying Kaggle download...
⚠️ Kaggle download failed: Command '['kaggle', 'competitions', 'download', '-c', 'ieee-fraud-detection', '-p', 'data']' returned non-zero exit status 1.
➡️  Generating small sample dataset instead (for demo only).
total 4.3M
-rw-r--r-- 1 root root  25K Oct 14 09:55 sample_submission.csv
-rw-r--r-- 1 root root 1.5M Oct 14 09:55 test_transaction.csv
-rw-r--r-- 1 root root 2.9M Oct 14 09:55 train_transaction.csv


In [33]:
import os, pandas as pd

DIR = "data/ieee"
has_identity = os.path.exists(f"{DIR}/train_identity.csv") and os.path.exists(f"{DIR}/test_identity.csv")

train_tr = pd.read_csv(f"{DIR}/train_transaction.csv")
test_tr  = pd.read_csv(f"{DIR}/test_transaction.csv")

if has_identity:
    train_id = pd.read_csv(f"{DIR}/train_identity.csv")
    test_id  = pd.read_csv(f"{DIR}/test_identity.csv")
    train = train_tr.merge(train_id, on="TransactionID", how="left")
    test  = test_tr.merge(test_id,  on="TransactionID", how="left")
else:
    # Fallback dataset has only transaction files
    train = train_tr.copy()
    test  = test_tr.copy()

label = "isFraud"
print("Train/Test shapes:", train.shape, test.shape)
print("Columns:", len(train.columns), "features")
if label in train.columns:
    print("Class balance:", train[label].value_counts(normalize=True))
else:
    raise ValueError("Label 'isFraud' not found in training data.")


Train/Test shapes: (5000, 32) (2500, 31)
Columns: 32 features
Class balance: isFraud
0    0.955
1    0.045
Name: proportion, dtype: float64


In [34]:
for df in (train, test):
    for c in df.select_dtypes(include=["float64"]).columns:
        df[c] = df[c].astype("float32")
print("Downcast done.")


Downcast done.


In [35]:
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(
    label=label, problem_type="binary", path="ag_ieee/"
).fit(
    train_data=train,
    presets="medium_quality_faster_train"  # try 'best_quality' later if you want
)
print("Training complete.")


Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       10.90 GB / 12.67 GB (86.0%)
Disk Space Avail:   62.53 GB / 112.64 GB (55.5%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ...
AutoGluon will save models to "/content/ag_ieee"
Train Data Rows:    5000
Train Data Columns: 31
Label Column:       isFraud
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11162.12 MB
	Train Data (Original)  Memory Usa

Training complete.


In [36]:
import os
os.makedirs("artifacts", exist_ok=True)

lb = predictor.leaderboard(silent=True)
fi = predictor.feature_importance(train)
lb.to_csv("artifacts/ieee_fraud_leaderboard.csv", index=False)
fi.to_csv("artifacts/ieee_fraud_feature_importance.csv", index=False)

lb.head(10), fi.head(10)


Computing feature importance via permutation shuffling for 31 features using 5000 rows with 5 shuffle sets...
	15.37s	= Expected runtime (3.07s per shuffle set)
	14.25s	= Actual runtime (Completed 5 of 5 shuffle sets)


(                 model  score_val eval_metric  pred_time_val   fit_time  \
 0  WeightedEnsemble_L2      0.986    accuracy       0.016934   7.232466   
 1      NeuralNetFastAI      0.984    accuracy       0.011784   5.355607   
 2             LightGBM      0.980    accuracy       0.003127   1.663465   
 3           LightGBMXT      0.980    accuracy       0.011377   6.286444   
 4             CatBoost      0.978    accuracy       0.001990   8.980087   
 5              XGBoost      0.978    accuracy       0.004328   1.794246   
 6       NeuralNetTorch      0.976    accuracy       0.005038   9.302696   
 7        LightGBMLarge      0.974    accuracy       0.022636   5.192815   
 8     RandomForestEntr      0.962    accuracy       0.074814   5.088484   
 9     RandomForestGini      0.962    accuracy       0.085479  10.993889   
 
    pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
 0                0.000822           0.082612            2       True   
 1              

In [38]:
probas = predictor.predict_proba(test)
# For binary classification, predict_proba returns a DataFrame with columns [0, 1]
p1 = probas[1] if hasattr(probas, "__getitem__") and 1 in getattr(probas, "columns", []) else probas

sub = pd.DataFrame({
    "TransactionID": test["TransactionID"].values if "TransactionID" in test.columns else range(len(test)),
    "isFraud": p1
})
sub.to_csv("artifacts/ieee_fraud_submission.csv", index=False)
sub.head()


,TransactionID,isFraud
0,1001430,0.000858
1,1000517,0.001264
2,1003069,0.001231
3,1003891,0.000167
4,1000751,0.001965


In [32]:
train_transaction = pd.read_csv('data/ieee/train_transaction.csv')
test_transaction  = pd.read_csv('data/ieee/test_transaction.csv')
...
predictor = TabularPredictor(label='isFraud', problem_type='binary', path='ag_ieee/').fit(...)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       10.90 GB / 12.67 GB (86.0%)
Disk Space Avail:   62.53 GB / 112.64 GB (55.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in competitions and

AssertionError: train_data is required to be a pandas DataFrame, but was instead: <class 'ellipsis'>

In [15]:
from google.colab import files
from pathlib import Path
import os

print("Upload kaggle.json (from your Kaggle account)...")
uploaded = files.upload()  # choose kaggle.json

filename = list(uploaded.keys())[0]  # automatically detect uploaded filename

Path("/root/.kaggle").mkdir(parents=True, exist_ok=True)
with open("/root/.kaggle/kaggle.json", "wb") as f:
    f.write(uploaded[filename])   # use actual filename instead of hardcoded one
os.chmod("/root/.kaggle/kaggle.json", 0o600)

!kaggle competitions list -s ieee | head -n 5




Upload kaggle.json (from your Kaggle account)...


Saving kaggle.json to kaggle (4).json
No competitions found


In [13]:
!pip -q install kaggle
!kaggle config view | sed -E 's/(api_key: ).+/\1[REDACTED]/'
!kaggle competitions list | head -n 15



Configuration values from /root/.kaggle
- username: anuradhasrivastav
- path: None
- proxy: None
- competition: None
ref                                                                              deadline             category                reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/arc-prize-2025                               2025-11-03 23:59:00  Featured         1,000,000 Usd       1119           False  
https://www.kaggle.com/competitions/jigsaw-agile-community-rules                 2025-10-24 06:59:00  Featured           100,000 Usd       2236           False  
https://www.kaggle.com/competitions/hull-tactical-market-prediction              2025-12-15 23:59:00  Featured           100,000 Usd       1021           False  
https://www.kaggle.com/competitions/google-code-golf-2025                

In [12]:
!kaggle config view | sed -E 's/(api_key: ).+/\1[REDACTED]/'

Configuration values from /root/.kaggle
- username: anuradhasrivastav
- path: None
- proxy: None
- competition: None
